<a href="https://colab.research.google.com/github/osmanatam/Blazor_applications/blob/master/AI_Platform_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is it?
This notebook demonstrates how to train a machine learning model with TensorFlow by using [AI Platform](https://cloud.google.com/ai-platform/), a serverless service by Google Cloud Platform for training and batch or online prediction. As this is a paid service, you may want to use [price calculator](https://cloud.google.com/products/calculator/) to estimate how much does your workload cost.

## By M. Yusuf Sarıgöz
I'm co-founder & AI engineer at [AI Labs](https://ailabs.com.tr), a Turkish startup revolutionizing industries with an AI-first approach, [Google Developer Expert on Machine Learning](https://developers.google.com/community/experts/directory/profile/profile-m._yusuf_sar_C4_B1g_C3_B6z), and lead at [TensorFlow Turkey](https://kommunity.com/tensorflow-turkey) community. You can follow me on [Twitter](https://twitter.com/myusufsarigoz), [GitHub](https://github.com/monatis) or connect me on [LinkedIn](https://www.linkedin.com/in/yusuf-sar%C4%B1g%C3%B6z-4bb826ba/).

## Getting Started
Below, you need to authenticate Google Cloud command line tools to access your Google Cloud services. To do so, run this cell, click on the link in the output preview, choose your Google Cloud account in the new tab, and finally copy and past the code below to complete the login process.

In [0]:
!gcloud auth login

## Set Project
Now, you need to set the id of the project you want to associate the resources with. You may need to create a new project and set the billing account on Google Cloud Console before running the command below. Then, replace `aiml-dev-2020` with your own project ID and run it.

In [0]:
!gcloud config set project aiml-dev-2020

## Enable Services
Run the three cells below to enable [Compute Engine](https://cloud.google.com/compute/), [Cloud Storage](https://cloud.google.com/products/storage/) and AI Platform (previously ML Engine) services. We will keep our dataset and models in Cloud Storage.

In [0]:
!gcloud services enable compute.googleapis.com

In [0]:
!gcloud services enable storage-component.googleapis.com

In [0]:
!gcloud services enable ml.googleapis.com

## Set Environment Variables
use `%env` magic to export some environment variables to use in subsequent  commands. If you want to replicate these commands in your workstation, you can use familiar `export` command to set environment variables.

Here, `RUNTIME_VERSION` refers to the version of the runtime that AI Platform will run our code on, and `PYTHON_VERSION` refers to the version of Python that AI Platform pass our code to. `REGION` may be of the regions supported by AI Platform, and `BUCKET_NAME` is the name of the Google Cloud Storage bucket to storage dataset in. A bucket with this name will be created further below. As any bucket name should be universally unique, please change it from `ailabscomtr19` to something else you own --a domain you manage, for example. Additionally, change the value of `PROJECT_ID` environment variable to the project ID you set previously under the `Set Project` section.

In [0]:
%env RUNTIME_VERSION=1.14

In [0]:
%env PYTHON_VERSION=3.5

In [0]:
%env REGION=us-central1

In [0]:
%env BUCKET_NAME=ailabscomtr19

In [0]:
%env PROJECT_ID=aiml-dev-2020

## Create Bucket
This command creates a bucket with the name you set in the previous section. If you try to run it multiple times, you may get an `Service Exception` stating that a bucket with name already exists --you may safely ignore it.

In [0]:
!gsutil mb gs://${BUCKET_NAME}

## Prepare Dataset
In this section, you will prepare the dataset to upload to Google Cloud Storage. In this sample, we will use [Cats And Dogs Filtered](https://bit.ly/33n6FcG), a small dataset for training purposes. It contains 1000 cat images and 1000 dogs images in the training set and 500 images for each one of the two categories in the validation set.


First, we will download it from a shortened URL and extract to a directory. Then we will set environment variables pointing where it resides in our local filesystem.

In [0]:
import tensorflow as tf
url = 'https://bit.ly/33n6FcG'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=url, extract=True)
path_to_zip

In [0]:
%env TRAIN_DIR=/root/.keras/datasets/cats_and_dogs_filtered/train

In [0]:
%env VAL_DIR=/root/.keras/datasets/cats_and_dogs_filtered/validation

## Upload Dataset
Set environment variables where we upload dataset on Google Cloud Storage. Change `ailabscomtr19` to your own bucket name that you created previously. Then, use `gsutil cp` command to upload the dataset.

In [0]:
%env GCS_TRAIN_DIR=gs://ailabscomtr19/datasets/cats_and_dogs/train

In [0]:
%env GCS_VAL_DIR=gs://ailabscomtr19/datasets/cats_and_dogs/validation

In [0]:
!gsutil -m cp -r ${TRAIN_DIR} ${GCS_TRAIN_DIR}

In [0]:
!gsutil -m cp -r ${VAL_DIR} ${GCS_VAL_DIR}

## Create Service Account
In this section, we will create a service account with necessary roles to manage AI Platform and Google Cloud Storage. We will store credentials in `key.json` file and then export `GOOGLE_APPLICATION_CREDENTIALS` environment variable poing to this file.

In [0]:
%env SERVICE_ACCOUNT_NAME=aiplatacc

In [0]:
!gcloud iam service-accounts create ${SERVICE_ACCOUNT_NAME} --display-name="Service Account for ai-platform"

In [0]:
!gcloud projects add-iam-policy-binding ${PROJECT_ID} --member serviceAccount:${SERVICE_ACCOUNT_NAME}@${PROJECT_ID}.iam.gserviceaccount.com --role roles/ml.developer

In [0]:
!gcloud projects add-iam-policy-binding ${PROJECT_ID} --member serviceAccount:${SERVICE_ACCOUNT_NAME}@${PROJECT_ID}.iam.gserviceaccount.com --role roles/storage.objectAdmin

In [0]:
%env GOOGLE_APPLICATION_CREDENTIALS=/content/key.json

In [0]:
!gcloud iam service-accounts keys create ${GOOGLE_APPLICATION_CREDENTIALS} --iam-account ${SERVICE_ACCOUNT_NAME}@${PROJECT_ID}.iam.gserviceaccount.com

## Clone Code Repo
Use `git` to clone the [repository](https://github.com/monatis/aiplatform-workshop) containing a very simple VGG-like model architecture and training logic. Please note that accuracy and performance is not a priority for the purpose of this sample, instead I aim at providing a concise code to focus more on the use of AI Platform.

In [0]:
!git clone https://github.com/monatis/aiplatform-workshop.git trainer

## Train Locally
Before running `trainer` in the cloud, train it one epoch locally  for sanity check. You may run this command multiple times, and it will not be charged.

In [0]:
!gcloud ai-platform local train --module-name=trainer.task --package-path=./trainer -- --train_dir ${TRAIN_DIR} --validation_dir ${VAL_DIR} --job-dir ./ --epochs 1

## Train in the Cloud
Finally, we will submit our training job to AI Platform to train the model in the cloud. To do so, we will export some environment variables to control where our training job will be executed.

Change `MODEL_DIR` to some path in your own GCS bucket. You may also want to update `MODEL_NAME` and `JOB_NAME` to reflect your own model. When the training job is running, you should be able to see logs streamed to the output preview. When you ge the log "Task completed successfully," you may understand that training is done and simply interrupt execution of the cell.

In [0]:
%env TIER=BASIC_GPU
%env MODEL_NAME="vgg8_cats_and_dogs"
%env PACKAGE_PATH=./trainer
%env MODEL_DIR=gs://ailabscomtr19/vgg8_cats_and_dogs/model
d=!date +%Y%m%d%H%M%S
d=str(d[0])
%env JOB_NAME=job_$d

In [0]:
!gcloud ai-platform jobs submit training ${JOB_NAME} --job-dir=${MODEL_DIR} --runtime-version=${RUNTIME_VERSION} --region=${REGION} --scale-tier=${TIER} --module-name=trainer.task --package-path=${PACKAGE_PATH} --python-version=${PYTHON_VERSION} --stream-logs -- --train_dir ${GCS_TRAIN_DIR} --validation_dir ${GCS_VAL_DIR} --epochs 5

## Conclusion
Congrats! You trained your first TensorFlow model in the cloud with AI Platform. Now, you may want to list files in `MODEL_DIR` by executing the cell below. You may download the model file to use for inference, or you may choose to serve it as a web API with AI Platform. I'm planning to publish a separate notebook about it. You may follow me on my social media accounts listed in the first section of this notebook to get updates about new notebooks.

In [0]:
!gsutil ls -r ${MODEL_DIR}/**